AWS RDS (MySQL) 에 Database 를 생성하고, 사용자 계정을 생성    

AWS RDS에서 데이터 크롤링하여 바로 입력하는 방식으로 진행해봄

### Starbucks Table
입력 테이블: COFFEE_STORE
- 데이터: 매장 이름, 매장이 위치한 구 이름, 매장 주소, 위도, 경도
- 필요한 데이터를 한세트씩 가져와서 COFFEE_STORE 테이블에 INSERT

In [ ]:
#!pip install mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import mysql.connector
import time
from bs4 import BeautifulSoup

# 구 리스트
gu_list = ['관악구','강북구','광진구','도봉구','구로구','중랑구','강남구','동작구','성동구','노원구','서초구','종로구','송파구','서대문구','용산구','동대문구','강동구','은평구','양천구','강서구', '마포구','성북구','금천구','중구','영등포구']

# MySQL 데이터베이스 연결
remote = mysql.connector.connect(
    host="database-1.cdo8s4228jzr.us-east-2.rds.amazonaws.com",
    port=3306,
    user='#####', # 암호화하였음
    password='###########', # 암호화하였음
    database='oneday' 
)

cur = remote.cursor(buffered=True)

# 스타벅스 페이지 접근
driver = webdriver.Chrome()

try:
    driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')
    time.sleep(5)

    # 설날 팝업 닫기
    try:
        driver.find_element(By.XPATH, '/html/body/div[4]/p/a').click()
    except NoSuchElementException:
        print('NoSuchElementException')

    # 지역 선택 클릭
    driver.find_element(By.CLASS_NAME, 'loca_search').click()
    time.sleep(2)

    # 서울 선택
    driver.find_element(By.XPATH, '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a').click()
    time.sleep(1)

    for gu in gu_list:
        driver.execute_script("window.scrollTo(0, 0);")
        driver.find_element(By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a').click()
        time.sleep(0.5)
        driver.find_element(By.XPATH, '''//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a''').click()
        time.sleep(0.5)
        if gu == '중랑구' or gu == '은평구' or gu == '종로구' or gu == '중구':
            scroll = driver.find_element(By.ID, 'mCSB_2_container')
            driver.execute_script('arguments[0].style.top = "-93px";', scroll)
            time.sleep(0.5)
        driver.find_element(By.LINK_TEXT, gu).click()
        time.sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        li_elements = soup.select('#mCSB_3_container > ul > li')
        storeNum = len(li_elements)        
    
        for i in range(storeNum):
            gu_name = gu
            name = li_elements[i].get('data-name')
            address = li_elements[i].find('p').text[:-9]
            latitude = li_elements[i].get('data-lat')
            longitude = li_elements[i].get('data-long')
            sql = "INSERT INTO COFFEE_STORE(brand, name, gu_name, address, lat, lng) VALUES (%s, %s, %s, %s, %s, %s)" 
            val = (1, name, gu_name, address, latitude, longitude)
            cur.execute(sql, val)
            remote.commit()
finally:
    driver.quit()


NoSuchElementException


### Ediya Table
입력 테이블: COFFEE_STORE
- 데이터: 매장 이름, 매장이 위치한 구 이름, 매장 주소, 위도, 경도
- 필요한 데이터를 한세트씩 가져와서 COFFEE_STORE 테이블에 INSERT
    - 단, ediya는 구가 '서울 중구' , '대구 중구' 와 같은 형식으로 되어있으므로 COFFEE_STORE에서 중복 제거 쿼리를 사용하여 가져와서 {'서울' + 구이름} 형식으로 변환하여 사용   
    - 최종 COFFEE_STORE 테이블에 입력할 구 이름은 {'서울'}이 제거든 구로 입력함

In [71]:
import googlemaps

gu_list = ['관악구','강북구','광진구','도봉구','구로구','중랑구','강남구','동작구','성동구','노원구','서초구','종로구','송파구','서대문구','용산구','동대문구','강동구','은평구','양천구','강서구', '마포구','성북구','금천구','중구','영등포구']

# MySQL 데이터베이스 연결
remote = mysql.connector.connect(
    host="database-1.cdo8s4228jzr.us-east-2.rds.amazonaws.com",
    port=3306,
    user='#####', # 암호화함
    password='############', #암호화함
    database='oneday'
)

cur = remote.cursor(buffered=True)

# 이디야 페이지 접근
driver = webdriver.Chrome()
driver.get('https://ediya.com/contents/find_store.html#c')
driver.find_element(By.CSS_SELECTOR, '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a').click()

gmaps_key = '#############' #암호화 함
gmaps = googlemaps.Client(key=gmaps_key)

for i in range(len(gu_list)):
    gu_list[i] = f'서울 {gu_list[i]}'

for i in range(len(gu_list)):
    driver.find_element(By.CSS_SELECTOR, '#keyword').send_keys(gu_list[i])
    driver.find_element(By.CSS_SELECTOR, '#keyword_div > form > button').click()
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(1)
    ediya = soup.select('#placesList > li')
    time.sleep(1)
    for store in ediya:
        gu_name = store.select_one('dd').text.split(" ")[1]
        name = store.select_one('dt').text
        address = store.select_one('dd').text

        # 위도 경도
        tmp = gmaps.geocode(address, language='ko')
        if tmp: 
            lat = tmp[0].get('geometry', {}).get('location', {}).get('lat', None)
            lng = tmp[0].get('geometry', {}).get('location', {}).get('lng', None)
        else:
            lat = None
            lng = None

        sql = "INSERT INTO COFFEE_STORE (brand, name, gu_name, address, lat, lng) VALUES (%s, %s, %s, %s, %s, %s)"
        val = (2, name, gu_name, address, lat, lng)
        cur.execute(sql, val)
        remote.commit()

    driver.find_element(By.CSS_SELECTOR, '#keyword').clear()
    time.sleep(1)

driver.quit()
remote.close()


### Tableau 시각화를 위한 CSV 파일 생성

- 전체 데이터를 가져오는데, 각 스타벅스 매장별로 이디야 전체 매장정보가 매칭 (정렬 : s_id, e_id 순)

In [15]:
import pandas as pd
import mysql.connector

# MySQL 데이터베이스 연결
remote = mysql.connector.connect(
    host="database-1.cdo8s4228jzr.us-east-2.rds.amazonaws.com",
    port=3306,
    user='#####', #암호화함
    password='##########', #암호화함
    database='oneday'
)

cur = remote.cursor(buffered=True)
query = "SELECT * FROM COFFEE_STORE"
cur.execute(query)

columns = [col[0] for col in cur.description]
data = cur.fetchall()
df = pd.DataFrame(data, columns=columns)
df.head()

In [20]:
starbucks = df.loc[df['brand']==1]
ediyas = df.loc[df['brand']==2]
starbucks = starbucks.rename(columns={'id':'s_id', 'brand':'s_brand', 'name':'s_name', 'gu_name':'s_gu', 'address':'s_address', 'lat':'s_lat', 'lng':'s_lng'})
ediyas = ediyas.rename(columns={'id':'e_id', 'brand':'e_brand', 'name':'e_name', 'gu_name':'e_gu', 'address':'e_address', 'lat':'e_lat', 'lng':'e_lng'})

merged_df = pd.merge(starbucks, ediyas, how='cross')
merged_df

,s_id,s_brand,s_name,s_gu,s_address,s_lat,s_lng,e_id,e_brand,e_name,e_gu,e_address,e_lat,e_lng
0,1,1,봉천역,관악구,서울특별시 관악구 남부순환로 1730 (봉천동),37.48203753400640,126.94228779253800,613,2,관악우성점,관악구,서울 관악구 관악로 232 (봉천동),37.48494770000000,126.95602740000000
1,1,1,봉천역,관악구,서울특별시 관악구 남부순환로 1730 (봉천동),37.48203753400640,126.94228779253800,614,2,관악하나로마트점,관악구,"서울 관악구 남부순환로 1369 (신림동, 관악농협농산물백화점)",37.48012740000000,126.90225710000000
2,1,1,봉천역,관악구,서울특별시 관악구 남부순환로 1730 (봉천동),37.48203753400640,126.94228779253800,615,2,낙성대역점,관악구,서울 관악구 남부순환로 1915 (봉천동),37.47792620000000,126.96222190000000
3,1,1,봉천역,관악구,서울특별시 관악구 남부순환로 1730 (봉천동),37.48203753400640,126.94228779253800,616,2,난곡사거리점,관악구,서울 관악구 남부순환로 1444 (신림동),37.48090380000000,126.91127540000000
4,1,1,봉천역,관악구,서울특별시 관악구 남부순환로 1730 (봉천동),37.48203753400640,126.94228779253800,617,2,난향동점,관악구,서울 관악구 난곡로 60 (신림동),37.46081360000000,126.91938330000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447367,612,1,타임스퀘어R,영등포구,서울특별시 영등포구 영중로 15 지하 1층 (영등포동4가),37.51755000000000,126.90516000000000,1339,2,영등포코레일점,영등포구,"서울 영등포구 경인로 846 (영등포동, 영등포 민자역사)",37.51592280000000,126.90737270000000
447368,612,1,타임스퀘어R,영등포구,서울특별시 영등포구 영중로 15 지하 1층 (영등포동4가),37.51755000000000,126.90516000000000,1340,2,영등포홈플러스점,영등포구,"서울 영등포구 당산로 42 (문래동3가, 홈플러스)",37.51898350000000,126.89870590000000
447369,612,1,타임스퀘어R,영등포구,서울특별시 영등포구 영중로 15 지하 1층 (영등포동4가),37.51755000000000,126.90516000000000,1341,2,한강성심병원점,영등포구,서울 영등포구 버드나루로 51 (영등포동2가),37.52060950000000,126.91219460000000
447370,612,1,타임스퀘어R,영등포구,서울특별시 영등포구 영중로 15 지하 1층 (영등포동4가),37.51755000000000,126.90516000000000,1342,2,한국수출입은행점,영등포구,"서울 영등포구 은행로 38 (여의도동, 한국수출입은행빌딩)",37.52860520000000,126.92339630000000


In [21]:
merged_df.to_csv('Starbucks&Ediya_maching.csv', index=False)